<a href="https://colab.research.google.com/github/abraham-30/ProjectStockPrediction/blob/main/stockprediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt

#DATA PROCESSING

In [ ]:
data_stock = pd.read_csv('BBCA.JK.csv')

##DATA VISUALIZATION